In [70]:
import os
import json
import gradio as gr
import anthropic
import base64
from io import BytesIO
from PIL import Image
from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import Audio, display
from datetime import datetime

In [12]:
# Initialization

# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

openai = OpenAI()

OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-
Google API Key exists and begins AIzaSyC0


In [3]:
system_message_assistant = "You are a helpful assistant for an Airline called FlightAI. "
system_message_assistant += "Give short, courteous answers, no more than 1 sentence. "
system_message_assistant += "Always be accurate. If you don't know the answer, say so."# Let's start by making a useful function

In [4]:
# Price function with corrosponding description for llms

ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")
    
price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [40]:
def book_a_flight(destination_city) :
    print ("flight is booked")
    return "Ticket reservation is a success, your flight will be on " + datetime.now().strftime("%Y-%m-%d %H:%M:%S")

book_a_flight_function = {
    "name": "book_a_flight",
    "description": "Books a flight to a destination city. Call this whenever you need to book a flight to a city, for example when a customer asks 'Please book a flight to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [30]:
# List of tools available for llms:

tools = [
    {"type": "function", "function": price_function},
    {"type": "function", "function": book_a_flight_function},
]

In [42]:
def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    response = {}
    city = ""
    if tool_call.function.name == 'get_ticket_price':  
        arguments = json.loads(tool_call.function.arguments)
        city = arguments.get('destination_city')
        price = get_ticket_price(city)
        response = {
            "role": "tool",
            "content": json.dumps({"destination_city": city,"price": price}),
            "tool_call_id": tool_call.id
        }
    elif tool_call.function.name == 'book_a_flight':
        arguments = json.loads(tool_call.function.arguments)
        city = arguments.get('destination_city')
        booking = book_a_flight(city)
        response = {
            "role": "tool",
            "content": json.dumps({"destination_city": city,"booking_time": booking}),
            "tool_call_id": tool_call.id
        }
   
    return response, city

In [15]:
def chat(history):
    MODEL="gpt-4o-mini"
    messages = [{"role": "system", "content": system_message_assistant}] + history
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    image = None
    
    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response, city = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        image = artist(city)
        response = openai.chat.completions.create(
            model=MODEL,
            messages=messages)
        
    reply = response.choices[0].message.content
    history += [{"role":"assistant", "content":reply}]

    talker(reply)
    
    return history, image

In [52]:
def translate(history):
    claude = anthropic.Anthropic()
    translation=[]
    for item in history:
        message = claude.messages.create(
            model="claude-3-haiku-20240307",
            max_tokens=1000,
            temperature=0.7,
            system="You are a professional translator for German. Always translates from English to German without adding any extra text.",
            messages=[
                {"role": "user", "content": item.get('content')},
            ],
        )
        translation+=[{"role":item.get('role'), "content":message.content[0].text}]
        
    return translation

In [9]:
def artist(city):
    image_response = openai.images.generate(
            model="dall-e-3",
            prompt=f"An image representing a vacation in {city}, showing tourist spots and everything unique about {city}, in a vibrant pop-art style",
            size="1024x1024",
            n=1,
            response_format="b64_json",
        )
    image_base64 = image_response.data[0].b64_json
    image_data = base64.b64decode(image_base64)
    return Image.open(BytesIO(image_data))

In [10]:
def talker(message):
    response = openai.audio.speech.create(
        model="tts-1",
        voice="alloy",
        input=message)

    audio_stream = BytesIO(response.content)
    output_filename = "output_audio.mp3"
    with open(output_filename, "wb") as f:
        f.write(audio_stream.read())

    # Play the generated audio
    display(Audio(output_filename, autoplay=True))

In [76]:
def talker(message):
    response = openai.audio.speech.create(
        model="tts-1",
        voice="alloy",
        input=message)

    audio_stream = BytesIO(response.content)
    output_filename = "output_audio.mp3"
    
    with open(output_filename, "wb") as f:
        f.write(audio_stream.read())
        
    display(Audio(output_filename, autoplay=True))

flight is booked


In [73]:
def transcribe(audio_path):
    with open(audio_path, "rb") as audio_file:
        transcription = openai.audio.transcriptions.create(
            model="whisper-1", 
            file=audio_file
        )
    return transcription.text


In [79]:
with gr.Blocks() as ui:
    with gr.Row():
        chatbot = gr.Chatbot(height=500, type="messages")
        chatbot_translate = gr.Chatbot(height=500, type="messages")
        image_output = gr.Image(height=500)
    with gr.Row():
        entry = gr.Textbox(label="Chat with our AI Assistant:")
        #audio_entry=gr.Microphone(type="filepath", label="Speak")
    with gr.Row():
        clear = gr.Button("Clear")

    def do_entry(message, history):
        history += [{"role":"user", "content":message}]
        return "", history

    entry.submit(
            do_entry, inputs=[entry, chatbot], outputs=[entry, chatbot]
        ).then(
            chat, inputs=chatbot, outputs=[chatbot, image_output]
        ).then(
            translate, inputs=chatbot, outputs=chatbot_translate
        )
        
    clear.click(lambda: None, inputs=None, outputs=chatbot, queue=False)

ui.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7895
* To create a public link, set `share=True` in `launch()`.


Tool get_ticket_price called for Berlin
